In [ ]:
import sys
import subprocess
import os
# Get the current working directory
current_folder = os.getcwd()
modules_path = os.path.join(current_folder, 'modules')
sys.path.append(current_folder)
sys.path.append(modules_path)

restart_command = [sys.executable] + sys.argv

# Call subprocess to restart the Python interpreter
subprocess.call(restart_command)

import logging
import pandas as pd

from modules.api_yf import api_yf
from modules.db_postgres_utils import db_utils
from modules.plotter import pl
from modules.processdata import processdata

# Define the ETF ticker symbol
ticker = 'META'
db_utils.connect()

# # Get a list of loaded modules
# loaded_modules = sys.modules.keys()
# # Print the list of loaded modules
# print(loaded_modules)


In [ ]:
data = db_utils.get_data_from_db(ticker)

data2 = processdata.add_daily_return_to_df(data)

start_date = processdata.formatDateDt(2021, 1, 1)
end_date = processdata.formatDateDt(2023, 1, 1)

# Calculate the return using the static method
total_return = processdata.get_return_defined_time(data2, start_date, end_date)


start_date = processdata.formatDateDt(2021, 1, 1)
end_date = processdata.formatDateDt(2024, 1, 1)

dStart, dEnd = processdata.get_valid_dates(data2, start_date, end_date)
roi = processdata.get_roi_defined_time(data2, dStart, dEnd)
mean = processdata.get_mean_between_dates(data2, dStart, dEnd)
std = processdata.get_sd_between_dates(data2, dStart, dEnd)
cov = processdata.get_cov_between_dates(data2, dStart, dEnd)

faang_list = ["META", "AMZN", "AAPL", "NFLX", "GOOG"]
mult_df = processdata.merge_df_by_column_name('daily_return',  '2020-1-1', '2020-12-31', *faang_list)

# Generate a Correlation Matrix
mult_df.corr()

# # We can look at the correlation between Netflix and the others
# mult_df.corr()['NFLX']

# # # We can plot this in a bar chart
mult_df.corr()['NFLX'].plot(kind='bar')

mult_df['NFLX'].var()

# Annualize by getting the number of samples and multiply
days = len(mult_df.index) # 253

nflx_a_var = mult_df['NFLX'].var() * 253
nflx_a_var

mult_df.corr()

In [ ]:
# get data from yf
ticker = 'NFLX'

data = api_yf.getFullDataFromYf(ticker)
# insert data into sqlLite db
db_utils.execute_values(data, 'stock_data', ticker)
db_utils.commit()

# retrive data from db
data = db_utils.get_data_from_db(ticker)
print(data.head())

# # Plot the closing prices over time using the Date column as x-axis
pl.plot_closing_prices(data, ticker)

In [ ]:
import yfinance as yf

# Get the ticker object for NASDAQ Composite Index
nasdaq_ticker = yf.Ticker("^IXIC")

# Get the list of tickers
nasdaq_symbols_list = nasdaq_ticker.tickers['^IXIC'].history().index.get_level_values(1).unique().tolist()

# Print the list of tickers
print(nasdaq_symbols_list)


In [ ]:

sec_df = df = pd.read_excel('c:\\Users\\somma\\Documents\\GIT\\pythonForFinance\\\\database\\wilshire_5000_stocks.xlsx')
db_utils.insert_wilshire_stocks(sec_df)
db_utils.commit()
sector = db_utils.get_wilshire_distinct_sectors()
tickers = db_utils.get_wilshire_ticker()
stockTicker = db_utils.get_ticker_from_stock()
print("wilshire tickers: ", len(tickers),"stock tickers : ",len(stockTicker))
unique_elements = [x for x in tickers if x not in stockTicker]
print(len(tickers)-len(stockTicker),len(unique_elements))
stocks_by_sector = db_utils.get_wilshire_ticker_by_sector("Industrials")

In [ ]:
tickers = db_utils.get_wilshire_ticker()

print("wilshire tickers: ", len(tickers),"stock tickers : ",len(stockTicker))
unique_elements = [x for x in tickers if x not in stockTicker]
print(len(tickers)-len(stockTicker),len(unique_elements))

In [ ]:
stockTicker = db_utils.get_ticker_from_stock()
import time

start_date = processdata.formatDateDt(2024, 6, 14)
end_date = processdata.formatDateDt(2024, 6, 23)

for ticker in stockTicker:
    df = api_yf.getFullDataFromYf(ticker)
    # insert data into db
    if df is not None and not df.empty:
        db_utils.execute_values(df, 'stock_data', ticker)
        db_utils.commit()
        time.sleep(3)


In [ ]:
# get data from yf
ticker = 'BNED'
start_date = processdata.formatDateDt(2024, 6, 14)
end_date = processdata.formatDateDt(2024, 6, 23)

data = api_yf.getFullDataFromYf(ticker)
# insert data into sqlLite db
db_utils.execute_values(data, 'stock_data', ticker)
db_utils.commit()

In [ ]:
import pandas as pd
import numpy as np

sectors = db_utils.get_wilshire_distinct_sectors()
print(sectors)

# Create an empty dictionary to store the dataframes
sector_dataframes = {}

# Loop through each sector and create a dataframe
for sector in sectors:
    sector_data = db_utils.get_wilshire_df_by_sector(sector)
    sector_dataframes[sector] = sector_data

def get_rois_for_stocks(stock_df,year):
    # Will hold all tickers & stock rois
    tickers = []
    rois = []

    # iterrows provides the index and column for each row in the DF
    for index, row in stock_df.iterrows():
        df = db_utils.get_data_from_db(row['ticker'])
        print('working on', row['ticker'],)
        # If we can't find the ticker delete it from the dataframe
        if df is None or len(df) < 1:
            pass
            print(row['ticker'], " is not available")
        else:          
            sdate, edate = processdata.get_valid_dates(df, f'{year}-01-01', f'{year}-12-31')
            if sdate != -1:
                roi = processdata.get_roi_defined_time(df, sdate, edate)
                tickers.append(row['ticker'])
                rois.append(roi)
    return pd.DataFrame({'ticker':tickers, 'ROI':rois})


# health_care = get_rois_for_stocks(health_df)
# it = get_rois_for_stocks(it_df)
# commun = get_rois_for_stocks(comm_df)
# staple = get_rois_for_stocks(staple_df)
# discretion = get_rois_for_stocks(discretion_df)
# utility = get_rois_for_stocks(utility_df)
# finance = get_rois_for_stocks(financial_df)
# material = get_rois_for_stocks(material_df)
# restate = get_rois_for_stocks(restate_df)
# energy = get_rois_for_stocks(energy_df)

In [ ]:
df = get_rois_for_stocks(sector_dataframes[sector], year)
df["sector"]=sector
db_utils.insert_stock_data(year,df)

In [ ]:
year = '2024'
df = db_utils.get_data_from_db('BNED')
sdate, edate = processdata.get_valid_dates(df, f'{year}-01-01', f'{year}-12-31')
if sdate != -1:
    roi = processdata.get_roi_defined_time(df, sdate, edate)
    print(roi)

db_utils.insert_stock_data(year,df)

In [ ]:
for year in range(2010, 2025):
    for sector in sectors:
        df = get_rois_for_stocks(sector_dataframes[sector], year)
        df["sector"]=sector
        db_utils.insert_stock_data(year,df)

    db_utils.commit()


In [ ]:
df = db_utils.get_stock_returns('2016','Technology')
df.sort_values(by=['return'], ascending=False).head(5)


In [ ]:
for year in range(2010, 2025):
    print(year)

In [ ]:
db_utils.rollback()

In [ ]:
thisDf = roisDict['Healthcare']
thisDf["sector"]='Healthcare'
print(thisDf.head())

In [ ]:
helatcare =roisDict['Healthcare']
helatcare.sort_values(by=['ROI'], ascending=False).head(5)

for el in roisDict:
    print("top 5 ", el)
    print(roisDict[el].sort_values(by=['ROI'], ascending=False).head(5))


In [ ]:
port_list = ["GNRC", "DXCM", "AMD", "NFLX", "COST", "TGT", "AES", "MSCI", 
             "NEM", "SBAC", "HES"]

mult_df = processdata.merge_df_by_column_name('daily_return',  '2021-01-04', 
                                  '2024-06-16', *port_list)

# Generate a Correlation Matrix
mult_df.corr()
# Get the number of samples
days = len(mult_df.index)
days

In [ ]:
import numpy as np
from math import ceil

date = '2021-01-04'

# Fetch initial values using a list comprehension and convert to NumPy array
initialValue = np.array([db_utils.get_data_from_db(stock, date, date)['adj_close'].values[0] for stock in port_list])
print("Initial Values:", initialValue)

# Calculate total investment and mean
tot_inv = 4 * np.sum(initialValue)
mean = tot_inv / len(initialValue)
print("Total Investment:", tot_inv, "Mean:", mean)

# Calculate stockNum using vectorized operations with NumPy
stockNum = np.ceil(mean / initialValue).astype(int)
print("Weights:", stockNum)

# Calculate the effectiveInvestment using NumPy arrays
effectiveInvestment = np.sum(initialValue * stockNum)
print("Result:", effectiveInvestment)

# Calculate the weights as an array of element-wise initialValue * stockNum / effectiveInvestment
weights = (initialValue * stockNum) / effectiveInvestment
print("Weight Calculations:", weights)

days = len(mult_df.index)

In [ ]:

port_var = np.dot(weights.T, np.dot(mult_df.cov() * days, weights))
print("Portfolio Covariance :", port_var)
print("GNRC Var :", mult_df["GNRC"].var() * days)
print("NFLX Var :", mult_df["NFLX"].var() * days)
print(mult_df.corr())

In [ ]:
# Diversifiable Risk = Portfolio Variance - All Weighted Variances
def calc_diversifiable_risk(df, tickers, weights):
    # Gets number of days
    days = len(mult_df.index)
    # Calculate covariance of portfolio
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights)) 
    
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = mult_df[tickers[i]].var() * days
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

div_risk = calc_diversifiable_risk(mult_df, port_list, weights)
div_risk

In [ ]:
# Find Risk that Can't be Diversified
print("Systematic Risk :", (port_var - div_risk))